<h1> Parameter Tuning
    
    Look for best learning rate 

# Importing libraries:

In [1]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn

# Import data; set X and y; fill nan values and split in test and training  data:

In [2]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# for later use
features_num = 15

# 2018 data
data = data.loc[data.index > 2018070000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values
data.fillna(method = 'ffill', inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

# Apply feature scaling:

In [3]:
from sklearn.preprocessing import MinMaxScaler

# data scaling  (including offer (y))
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Importing the Keras libraries and packages:

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

mae_gen = []
rmse_gen = []
mae_nor = []
mae_spi = []
rmse_nor = []
rmse_spi = []
hist_list = []
y_pred_list = []
prediction_list = []
time_count = []

# Prepare of data according to LSTM needs,  create regressor & tune:

In [5]:
# parameters
steps = 96
n_hidden = 1
units = 150
batch_size = 96

# parameters:
learning_range =  [0.0001, 0.001, 0.01, 0.1, 1]


# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# divide features and labels
X_train = data_train[:, 0:15] 
y_train = data_train[:, -1]
X_test = data_test[:, 0:15] 
y_test = data_test[:, -1] 

# divide data into train and test 
X_train, X_val, y_train, y_val = train_test_split(
         X_train, y_train, test_size = 0.15, shuffle=False)

# put data into correct shape
X_train, y_train = split_data(X_train, y_train, steps)
X_test, y_test = split_data(X_test, y_test, steps)
X_val, y_val = split_data(X_val, y_val, steps)

X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

# inverse of test set should not be inside the loop 
y_test = (y_test * sc_X.data_range_[15]) + (sc_X.data_min_[15])
               
for i in learning_range:
    
    # design the LSTM
    def regressor_tunning(kernel_initializer = 'he_normal',
                          bias_initializer = initializers.Ones()):

        model = Sequential()

        if n_hidden == 0:
            model.add(LSTM(units = units,                    
                           batch_input_shape = (batch_size, steps, features_num), 
                           stateful = True,
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
        else:
            model.add(LSTM(units = units,                    
                           batch_input_shape = (batch_size, steps, features_num), 
                           stateful = True,
                           return_sequences = True,
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
            model.add(LSTM(units = units, 
                           batch_input_shape = (batch_size, steps, features_num), 
                           stateful = True,
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
        model.add(Dense(1, activation='linear'))
        optimizer = optimizers.RMSprop(lr = i)
        model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
        return model

    model = regressor_tunning()

    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = 180,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15
    y_pred = (y_pred * sc_X.data_range_[15]) + (sc_X.data_min_[15])

    y_pred_list.append(y_pred)
    
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)
    
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)
    

Epoch 1/180
65/65 [==============================] - 30s 466ms/step - loss: 0.2253 - mse: 0.2253 - mae: 0.3684 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0732
Epoch 2/180
65/65 [==============================] - 31s 473ms/step - loss: 0.1370 - mse: 0.1370 - mae: 0.2954 - val_loss: 0.0173 - val_mse: 0.0173 - val_mae: 0.1241
Epoch 3/180
65/65 [==============================] - 32s 489ms/step - loss: 0.0944 - mse: 0.0944 - mae: 0.2456 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0379
Epoch 4/180
65/65 [==============================] - 33s 505ms/step - loss: 0.0626 - mse: 0.0626 - mae: 0.1992 - val_loss: 0.0028 - val_mse: 0.0028 - val_mae: 0.0455
Epoch 5/180
65/65 [==============================] - 32s 500ms/step - loss: 0.0416 - mse: 0.0416 - mae: 0.1631 - val_loss: 0.0104 - val_mse: 0.0104 - val_mae: 0.0965
Epoch 6/180
65/65 [==============================] - 32s 489ms/step - loss: 0.0280 - mse: 0.0280 - mae: 0.1326 - val_loss: 0.0061 - val_mse: 0.0061 - val_mae: 0.0736
Epoc

Epoch 96/180
65/65 [==============================] - 36s 553ms/step - loss: 8.8561e-04 - mse: 8.8561e-04 - mae: 0.0172 - val_loss: 9.8986e-04 - val_mse: 9.8986e-04 - val_mae: 0.0175
Epoch 97/180
65/65 [==============================] - 42s 645ms/step - loss: 8.9463e-04 - mse: 8.9463e-04 - mae: 0.0172 - val_loss: 9.8615e-04 - val_mse: 9.8615e-04 - val_mae: 0.0174
Epoch 98/180
65/65 [==============================] - 41s 634ms/step - loss: 8.9638e-04 - mse: 8.9638e-04 - mae: 0.0172 - val_loss: 9.8680e-04 - val_mse: 9.8680e-04 - val_mae: 0.0176
Epoch 99/180
65/65 [==============================] - 39s 599ms/step - loss: 8.9393e-04 - mse: 8.9393e-04 - mae: 0.0172 - val_loss: 9.8311e-04 - val_mse: 9.8311e-04 - val_mae: 0.0174
Epoch 100/180
65/65 [==============================] - 37s 565ms/step - loss: 8.8750e-04 - mse: 8.8750e-04 - mae: 0.0172 - val_loss: 9.8277e-04 - val_mse: 9.8277e-04 - val_mae: 0.0177
Epoch 101/180
65/65 [==============================] - 40s 610ms/step - loss: 8.8688

Epoch 5/180
65/65 [==============================] - 60s 917ms/step - loss: 0.0043 - mse: 0.0043 - mae: 0.0508 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0178
Epoch 6/180
65/65 [==============================] - 55s 841ms/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0345 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0251
Epoch 7/180
65/65 [==============================] - 55s 843ms/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0259 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0197
Epoch 8/180
65/65 [==============================] - 60s 925ms/step - loss: 0.0012 - mse: 0.0012 - mae: 0.0216 - val_loss: 0.0013 - val_mse: 0.0013 - val_mae: 0.0170
Epoch 9/180
65/65 [==============================] - 61s 939ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0200 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0164
Epoch 10/180
65/65 [==============================] - 57s 883ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0192 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0164
Epo

Epoch 99/180
65/65 [==============================] - 45s 700ms/step - loss: 9.0642e-04 - mse: 9.0642e-04 - mae: 0.0175 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0182
Epoch 100/180
65/65 [==============================] - 45s 698ms/step - loss: 9.0377e-04 - mse: 9.0377e-04 - mae: 0.0175 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0182
Epoch 101/180
65/65 [==============================] - 47s 718ms/step - loss: 9.0184e-04 - mse: 9.0184e-04 - mae: 0.0175 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0182
Epoch 102/180
65/65 [==============================] - 46s 700ms/step - loss: 9.0319e-04 - mse: 9.0319e-04 - mae: 0.0175 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0183
Epoch 103/180
65/65 [==============================] - 46s 704ms/step - loss: 8.9946e-04 - mse: 8.9946e-04 - mae: 0.0175 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0181
Epoch 104/180
65/65 [==============================] - 46s 711ms/step - loss: 9.0155e-04 - mse: 9.0155e-04 - mae: 0.0175 

Epoch 12/180
65/65 [==============================] - 46s 711ms/step - loss: 9.3911e-04 - mse: 9.3911e-04 - mae: 0.0182 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0163
Epoch 13/180
65/65 [==============================] - 47s 717ms/step - loss: 9.3911e-04 - mse: 9.3911e-04 - mae: 0.0182 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0163
Epoch 14/180
65/65 [==============================] - 48s 740ms/step - loss: 9.3910e-04 - mse: 9.3910e-04 - mae: 0.0182 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0163
Epoch 15/180
65/65 [==============================] - 46s 709ms/step - loss: 9.3910e-04 - mse: 9.3910e-04 - mae: 0.0182 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0163
Epoch 16/180
65/65 [==============================] - 48s 736ms/step - loss: 9.3909e-04 - mse: 9.3909e-04 - mae: 0.0182 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0163
Epoch 17/180
65/65 [==============================] - 49s 759ms/step - loss: 9.3910e-04 - mse: 9.3910e-04 - mae: 0.0182 - val

...

Epoch 166/180
65/65 [==============================] - 52s 796ms/step - loss: 9.3159 - mse: 9.3159 - mae: 3.0370 - val_loss: 8.2071 - val_mse: 8.2071 - val_mae: 2.8646
Epoch 167/180
65/65 [==============================] - 51s 790ms/step - loss: 9.1820 - mse: 9.1820 - mae: 3.0134 - val_loss: 9.4739 - val_mse: 9.4739 - val_mae: 3.0778
Epoch 168/180
65/65 [==============================] - 52s 793ms/step - loss: 9.4161 - mse: 9.4161 - mae: 3.0566 - val_loss: 8.1156 - val_mse: 8.1156 - val_mae: 2.8486
Epoch 169/180
65/65 [==============================] - 52s 806ms/step - loss: 9.1926 - mse: 9.1926 - mae: 3.0180 - val_loss: 8.9084 - val_mse: 8.9084 - val_mae: 2.9845
Epoch 170/180
65/65 [==============================] - 52s 803ms/step - loss: 9.5001 - mse: 9.5001 - mae: 3.0582 - val_loss: 8.8724 - val_mse: 8.8724 - val_mae: 2.9785
Epoch 171/180
65/65 [==============================] - 51s 787ms/step - loss: 9.2896 - mse: 9.2896 - mae: 3.0291 - val_loss: 7.2129 - val_mse: 7.2129 - val_mae:

Epoch 33/180
65/65 [==============================] - 52s 797ms/step - loss: 998.9583 - mse: 998.9583 - mae: 31.4305 - val_loss: 858.3569 - val_mse: 858.3569 - val_mae: 29.2963
Epoch 34/180
65/65 [==============================] - 52s 798ms/step - loss: 994.9581 - mse: 994.9581 - mae: 31.4180 - val_loss: 891.9218 - val_mse: 891.9218 - val_mae: 29.8643
Epoch 35/180
65/65 [==============================] - 52s 799ms/step - loss: 995.4257 - mse: 995.4257 - mae: 31.4041 - val_loss: 901.1198 - val_mse: 901.1198 - val_mae: 30.0182
Epoch 36/180
65/65 [==============================] - 53s 809ms/step - loss: 999.1699 - mse: 999.1699 - mae: 31.4605 - val_loss: 876.4672 - val_mse: 876.4672 - val_mae: 29.6040
Epoch 37/180
65/65 [==============================] - 52s 799ms/step - loss: 991.7324 - mse: 991.7324 - mae: 31.3571 - val_loss: 1031.1510 - val_mse: 1031.1510 - val_mae: 32.1113
Epoch 38/180
65/65 [==============================] - 53s 811ms/step - loss: 1007.0225 - mse: 1007.0225 - mae: 31

Epoch 79/180
65/65 [==============================] - 53s 809ms/step - loss: 994.5077 - mse: 994.5077 - mae: 31.3859 - val_loss: 888.8495 - val_mse: 888.8495 - val_mae: 29.8135
Epoch 80/180
65/65 [==============================] - 52s 799ms/step - loss: 1001.8018 - mse: 1001.8018 - mae: 31.5233 - val_loss: 958.3121 - val_mse: 958.3121 - val_mae: 30.9557
Epoch 81/180
65/65 [==============================] - 52s 795ms/step - loss: 1000.0753 - mse: 1000.0753 - mae: 31.4313 - val_loss: 950.5861 - val_mse: 950.5861 - val_mae: 30.8312
Epoch 82/180
65/65 [==============================] - 50s 777ms/step - loss: 994.7231 - mse: 994.7231 - mae: 31.4336 - val_loss: 963.9595 - val_mse: 963.9595 - val_mae: 31.0475
Epoch 83/180
65/65 [==============================] - 51s 786ms/step - loss: 999.3581 - mse: 999.3581 - mae: 31.4722 - val_loss: 947.7867 - val_mse: 947.7867 - val_mae: 30.7860
Epoch 84/180
65/65 [==============================] - 52s 794ms/step - loss: 986.9896 - mse: 986.9896 - mae: 31

Epoch 125/180
65/65 [==============================] - 53s 813ms/step - loss: 995.5456 - mse: 995.5456 - mae: 31.4280 - val_loss: 1161.0590 - val_mse: 1161.0590 - val_mae: 34.0743
Epoch 126/180
65/65 [==============================] - 52s 796ms/step - loss: 1001.4174 - mse: 1001.4174 - mae: 31.5055 - val_loss: 920.1736 - val_mse: 920.1736 - val_mae: 30.3343
Epoch 127/180
65/65 [==============================] - 52s 799ms/step - loss: 999.1029 - mse: 999.1029 - mae: 31.4893 - val_loss: 770.5702 - val_mse: 770.5702 - val_mae: 27.7578
Epoch 128/180
65/65 [==============================] - 52s 801ms/step - loss: 996.1179 - mse: 996.1179 - mae: 31.3966 - val_loss: 1036.6444 - val_mse: 1036.6444 - val_mae: 32.1967
Epoch 129/180
65/65 [==============================] - 53s 810ms/step - loss: 984.5162 - mse: 984.5162 - mae: 31.2182 - val_loss: 955.0888 - val_mse: 955.0888 - val_mae: 30.9041
Epoch 130/180
65/65 [==============================] - 51s 779ms/step - loss: 1004.0728 - mse: 1004.0727

Epoch 171/180
65/65 [==============================] - 53s 813ms/step - loss: 1000.1096 - mse: 1000.1094 - mae: 31.4761 - val_loss: 1067.3875 - val_mse: 1067.3875 - val_mae: 32.6699
Epoch 172/180
65/65 [==============================] - 53s 816ms/step - loss: 994.2800 - mse: 994.2800 - mae: 31.3946 - val_loss: 1057.3909 - val_mse: 1057.3909 - val_mae: 32.5174
Epoch 173/180
65/65 [==============================] - 53s 808ms/step - loss: 997.6727 - mse: 997.6727 - mae: 31.4377 - val_loss: 1054.2012 - val_mse: 1054.2012 - val_mae: 32.4682
Epoch 174/180
65/65 [==============================] - 51s 783ms/step - loss: 992.0836 - mse: 992.0836 - mae: 31.3356 - val_loss: 960.5771 - val_mse: 960.5771 - val_mae: 30.9916
Epoch 175/180
65/65 [==============================] - 50s 774ms/step - loss: 1004.5898 - mse: 1004.5898 - mae: 31.5382 - val_loss: 1103.0768 - val_mse: 1103.0768 - val_mae: 33.2125
Epoch 176/180
65/65 [==============================] - 51s 792ms/step - loss: 991.3817 - mse: 991.

In [7]:
results = pd.DataFrame({                      
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor}, index = learning_range)

results.to_csv('Results_LSTM_4_learning_rate.csv')

results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0.0001,32.601976,20.337302,29.907247,19.861724,32.983513,20.407951
0.0010,33.070140,22.769643,29.792108,22.155887,33.529777,22.860819
0.0100,33.846575,17.150002,29.830015,15.918361,34.403270,17.332968
0.1000,4404.038809,4403.913695,4400.573603,4400.473635,4404.553347,4404.424731
1.0000,53658.562148,53658.551879,53655.120017,53655.111818,53659.073472,53659.062915


In [8]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0.0001,32.601976,20.337302,29.907247,19.861724,32.983513,20.407951
0.001,33.070140,22.769643,29.792108,22.155887,33.529777,22.860819
0.01,33.846575,17.150002,29.830015,15.918361,34.403270,17.332968
0.1,4404.038809,4403.913695,4400.573603,4400.473635,4404.553347,4404.424731
1.0,53658.562148,53658.551879,53655.120017,53655.111818,53659.073472,53659.062915


In [ ]:
!pip install matplotlib

import matplotlib.pyplot as plt